In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import re, sys, random, glob, json, unidecode, unicodedata
import ipdb, time, sys, os, pickle
import requests 
from IPython.core.display import display, HTML
from IPython.display import clear_output
from urllib.request import urlopen
from collections import Counter
from bs4 import BeautifulSoup  

display(HTML("<style>.container { width:100% !important; }</style>"))

# Matplotlib Configuration
pylab.rcParams.update({'legend.fontsize': 'medium',
                       'font.size'      : 12,
                       'figure.figsize' : (10, 10),
                       'axes.labelsize' : 'medium',
                       'axes.titlesize' : 'medium',
                       'axes.grid'      : 'on',
                       'xtick.labelsize': 'medium',
                       'ytick.labelsize': 'medium'})

def print_runtime(start: float, printer: bool = True) -> None:
    end = time.time()
    if printer:
        print(f'Runtime: {int((end - start) // 60)} min {int((end - start) % 60):2d} sec')
    else:
        return f'({int((end - start) // 60)} min {int((end - start) % 60):2d} sec)'


with open('urls.txt', 'r') as f:
    urls = f.readlines()
urls = [url[:-1] for url in urls]

/var/folders/jf/qykqz6ks0vs56cb_zyf_7x_00000gn/T/ipykernel_73755/1860089765.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [233]:
from openai import OpenAI
start = time.time()
client = OpenAI()
model = "gpt-3.5-turbo"
bios = []
seen = set()


for iu, url in enumerate(urls[114:]):
    print(f'\n{iu} or {len(urls)} main URL: {url}')
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        a_tags = soup.find_all("a") #
    except Exception:
        print(f'    Exception at main url: {url} {Exception}')
        continue
    
    bios.append({'URL':url, 'estimate':None})
    
    for idx, a_tag in enumerate(a_tags):
        href = a_tag.get("href")
        if not urlparse(href).hostname:
            href = url + urlparse(href).geturl()
        if urlparse(href).hostname is not None and \
            not urlparse(href).hostname.endswith(urlparse(url).hostname) and not urlparse(url).hostname.endswith(urlparse(href).hostname) :
            print(f'        hostname mismatch: {urlparse(href).hostname} != {urlparse(url).hostname}')
            continue
        if href in seen:
            #print('    seen')
            continue
        
        print(f'    {idx} of {len(a_tags)-1} ', end=' ')
        seen.add(href)

        for trial in range(5):
            prompt = """I'll give you an HTML tag that's extracted from the web page of a company. I want you to estimate the likelihood """ +\
            """that this tag leads to page that's named something like "About the Team" or "About Us" or "Our Team" or """+\
            """it leads to a Bio page of any of the employees in the company. """ +\
            """\nProvide your estimate as a single integer number between 0 and 100, where 0 means very unlikely, """ +\
            """and 100 means absolutely certain. Do not include any other text or explanation in your response. \n""" +\
            """Here's the HTML tag: """ + str(a_tag)

            completion = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that answers only and only in integer numeric values"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.01,
                logprobs=True
            )

            content = completion.choices[0].message.dict()['content']
            estimate = int(float(content))

        if estimate >= 40:
            bios.append({"URL":href, "estimate":estimate})

        print(f'estimate = {estimate} {href} ')
    print(f'    len(bios) {len(bios)} {print_runtime(start, False)}')
    

In [61]:
def extract_text_from_url(url, data, visited_urls):
    if url in visited_urls:
        print(f'url already in visited_urls: {url}')
        return None, None
    
    try:
        html = urlopen(url).read()
    except Exception as e:
        stdout = f'{idx:5d}  url: {url}  I had an exception {e}'
        print(stdout)
        visited_urls.add(url)
        data[url] = stdout
        return None, None
    
    
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    data[url] = text
    visited_urls.add(url)
    print(f'{idx:5d}  {len(data[url])} chars')
    return html, text

data = dict()
visited_urls = set()
for idx, url in enumerate(urls[1100:], start=1100):
    if idx in {1100,}:
        continue
    html, text = extract_text_from_url(url, data, visited_urls)
    time.sleep(.1)


 1101  5922 chars
 1102  4263 chars
 1103  6957 chars
 1104  3326 chars
 1105  url: https://www.business-acquisitions.com/  I had an exception HTTP Error 403: Forbidden
 1106  3525 chars
url already in visited_urls: https://www.business-team.com/
url already in visited_urls: https://www.business-team.com/
url already in visited_urls: https://www.business-team.com/
url already in visited_urls: https://www.business-team.com/
 1111  12103 chars
 1112  2628 chars
 1113  3726 chars
 1114  1876 chars
 1115  3778 chars
 1116  9111 chars
 1117  2837 chars
 1118  url: https://www.bxbbrokers.com/  I had an exception HTTP Error 406: Not Acceptable
 1119  url: https://www.caldergr.com/  I had an exception HTTP Error 403: Forbidden
 1120  5834 chars
 1121  1418 chars
 1122  url: https://www.capitalconnectus.com/  I had an exception HTTP Error 403: Forbidden
 1123  url: https://www.cibb.com/  I had an exception HTTP Error 403: Forbidden
 1124  url: https://www.citizensbusinessgroup.com/  I had an ex

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


 1182  72192 chars
 1183  2042 chars
 1184  url: https://www.lbaweb.com/  I had an exception HTTP Error 403: Forbidden
 1185  7391 chars
 1186  url: https://www.linkedin.com/company/iowa-business-advisers/  I had an exception HTTP Error 999: Request denied
 1187  url: https://www.linkedin.com/in/daniel-popovich/  I had an exception HTTP Error 999: Request denied
 1188  url: https://www.linkenterprisesv.com/  I had an exception HTTP Error 403: Forbidden
 1189  url: https://www.lionbusinessbrokers.com/  I had an exception HTTP Error 403: Forbidden
 1190  url: https://www.liveoakbank.com/  I had an exception HTTP Error 403: Forbidden
 1191  6681 chars
 1192  3155 chars
 1193  url: https://www.marcelobermudezinc.com/  I had an exception HTTP Error 406: Not Acceptable
 1194  url: https://www.mbcloans.biz/  I had an exception HTTP Error 403: Forbidden
 1195  4375 chars
 1196  24878 chars
 1197  url: https://www.midmarketbusinesses.com/  I had an exception HTTP Error 403: Forbidden
 1198  469

In [64]:
# with open('scraped_data_3.json', 'w') as f:
#     json.dump(data, f)

In [4]:
def get_all_links(url):
    all_links = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    a_tags = soup.find_all("a") #
    for a_tag in a_tags:
        new_link = a_tag.get("href")
        if new_link.startswith('#'):
            continue
        if new_link.startswith('/'):
            new_link = url + new_link

        all_links.append(new_link)

    return all_links

def get_team_links(all_links):
    team_links = [link for link in all_links if 'team' in link]
    return team_links

url = 'http://baystatebusinessbrokers.com/' # 'https://ibgbusiness.com/'
all_links = get_all_links(url)
team_links = get_team_links(all_links)
team_links


[]

In [202]:
url = '#'
r = urlparse(url)
r.scheme
r.netloc
r.path
r

ParseResult(scheme='', netloc='', path='', params='', query='', fragment='')

In [195]:
url = 'https://hardingbusinessfl.com/about/#who-about'
dir(urlparse(url))

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_encoded_counterpart',
 '_field_defaults',
 '_fields',
 '_fields_defaults',
 '_hostinfo',
 '_make',
 '_replace',
 '_userinfo',
 'count',
 'encode',
 'fragment',
 'geturl',
 'hostname',
 'index',
 'netloc',
 'params',
 'password',
 'path',
 'port',
 'query',
 'scheme',
 'username']

In [139]:
from urllib.parse import urlparse
r = (urlparse(url))
print(url)
print(r.scheme)
print(r.hostname)
print(r.path)


https://www.acquisitionexperts.net/florida-lawn-care-landscaping-business-listings/
https
www.acquisitionexperts.net
/florida-lawn-care-landscaping-business-listings/
